In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam

/var/folders/_x/1j3rm6sd6cn3kd7qpmy3s_k40000gn/T/ipykernel_38144/3642130788.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


# CSV Data: Random Forest + XGBoost 

### Pre- Processing Data

In [2]:
# Loading dataset
df = pd.read_excel('/Users/sukanya/Documents/Alziemers Multi Modal/Dataset/FINAL.xlsx')

X = df.drop(columns=['ID', 'Dementia_Category'])
y = df['Dementia_Category']

# Spliting data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scaling features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


### Model

In [3]:
from sklearn.ensemble import VotingClassifier
import xgboost as xgb

# Voting Classifier with Random Forest and XGBoost
xgb_clf = xgb.XGBClassifier()
rf_xgb_model = VotingClassifier(
    estimators=[('rf', RandomForestClassifier()), ('xgb', xgb_clf)],
    voting='soft'
)

rf_xgb_model.fit(X_train_scaled, y_train)

VotingClassifier(estimators=[('rf', RandomForestClassifier()),
                             ('xgb',
                              XGBClassifier(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None, device=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric=None,
                                            feature_types=None, gamma=None,
                                            grow_policy=None,
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learning_rate=None, max_bin=None,
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=None,
                                            max_leaves=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            multi_strategy=None,
                                            n_estimators=None, n_jobs=None,
                                            num_parallel_tree=None,
                                            random_state=None, ...))],
                 voting='soft')

# Image Data: Xception

### Loading Images

In [4]:
# Paths
train_dir = '/Users/sukanya/Documents/Alziemers Multi Modal/Split Dataset/train'  
test_dir = '/Users/sukanya/Documents/Alziemers Multi Modal/Split Dataset/test'

#  Loading images and label
train_dataset = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    image_size=(256, 256),  
    batch_size=32,         
    label_mode='int',       
    shuffle=True
)

test_dataset = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    image_size=(256, 256),
    batch_size=32,
    label_mode='int',
    shuffle=True
)

# Class Names
class_names = train_dataset.class_names
print("Class names:", class_names)


Found 795 files belonging to 4 classes.
Found 201 files belonging to 4 classes.
Class names: ['Mild Dementia', 'Moderate Dementia', 'No Dementia', 'Severe Dementia ']


### Normalizing Images

In [5]:
# Normalize the image data
normalization_layer = layers.Rescaling(1./255)

train_dataset = train_dataset.map(lambda x, y: (normalization_layer(x), y))
test_dataset = test_dataset.map(lambda x, y: (normalization_layer(x), y))

### Model

In [ ]:
from tensorflow.keras.applications import Xception
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

In [ ]:
# Base Model
base_model = Xception(weights='imagenet', include_top=False)
for layer in base_model.layers:
    layer.trainable = False

model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(256, activation='relu'))
model.add(Dense(4, activation='softmax'))

model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

83683744/83683744 [==============================] - 124s 1us/step


In [4]:
xception_model = tf.keras.models.load_model('/Users/sukanya/Documents/Alziemers Multi Modal/MRI Images/Xception_model.h5')

# Late Fusion of Predictions from both Models

In [5]:
# Preprocessing and predicting for a single MRI image
def preprocess_image(img_path):
    img = load_img(img_path, target_size=(256, 256)) 
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    return img_array / 255.0 

In [6]:
# Late fusion function
def late_fusion(rf_xgb_probs, xception_probs, weight_csv=0.6, weight_img=0.4):
    combined_probs = (weight_csv * rf_xgb_probs) + (weight_img * xception_probs)
    final_class = np.argmax(combined_probs, axis=1)
    return final_class

In [ ]:
# Testing a single image and numerical entry
category_mapping = {
    0: "Mild Dementia",
    1: "Moderate Dementia",
    2: "No Dementia",
    3: "Severe Dementia"
}
def test(rf_xgb_model, xception_model, scaled_data, img_path):
    # Model predictions
    rf_xgb_probs = rf_xgb_model.predict_proba(scaled_data)  # numerical data
    
    # Preprocessing the image and get Xception predictions
    img_array = preprocess_image(img_path)
    xception_probs = xception_model.predict(img_array)  # MRI image data
    
    # Late fusion
    final_class = late_fusion(rf_xgb_probs, xception_probs)
    
    # Retrieving class
    category_name = category_mapping[final_class[0]]
    print(f"Predicted Dementia Category: {final_class[0]} - {category_name}")

In [8]:
numerical_data = np.array([[0, 0, 74, 2	, 3, 29, 0,	1344, 0.743,	1.306,	20.55]]) 
img_path = '/Users/sukanya/Documents/Alziemers Multi Modal/Test Data/Test Images/Test1.jpg'

scaled_data = scaler.transform(numerical_data)

/opt/homebrew/lib/python3.11/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [9]:
# Testing
test(rf_xgb_model, xception_model, scaled_data, img_path)

1/1 [==============================] - 0s 283ms/step
Predicted Dementia Category: 2 - No Dementia
